This Kaggle dataset is about customer conversion on Google's Google Merchandise Store (also known as GStore, where Google swag is sold). The main purpose of this analysis is to predict revenue per customer and make recommandations on promotional strategies. The main technical challenge it poses to predicting revenue is the presence of multiple high cardinality categorical features. By careful data exploration followed by well-thought choice of feature treatments as well as machine learning algorithm, I show that an optimal solution based on feature-engineering and extreme gradient-boosted decision trees yields an enhanced predictive power of 0.997, as measured by the area under the precision-recall curve. Crucially, these results were obtained without artificial balancing of the data making this approach suitable to real-world applications.

<a id='top'></a>
#### Outline: 
#### 0. <a href='#Sampling'>Random sampling of training set</a>


#### 1. <a href='#clean'>Data Cleaning</a>
11. <a href='#cleanID'>Cleaning of IDs</a>
12. <a href='#cleanTotals'>Cleaning of Variable Totals</a>
13. <a href='#cleanTS'>Cleaning of Time Series Variables</a>
14. <a href='#cleanLoc'>Cleaning of Location Variables</a>
15. <a href='#cleanDev'>Cleaning of Device Variables</a>
16. <a href='#cleanCust'>Cleaning of Custom Dimension Variables</a>


#### 2. <a href='#feature-eng'>Feature Engineering</a>
21. <a href='#dropUnif'> Drop uninformative categorical varibles</a>
22. <a href='#dropMissing'>Drop variables with too many missing values</a>
23. <a href='#encodeCat'>Encode Categorical Variables</a>
24. <a href='#encodeCat1'>Encode Network Domain</a>
25. <a href='#encodeCat2'>Encode Operating Systems</a>

#### 3. <a href='#EDA'>Exploratory Data Analysis</a>

#### 4. <a href='#ML'>Machine Learning to Predict Transactions</a>
41. <a href='#rfEnum'>Random Forest with Enum Encoding</a>
42. <a href='#gbmEnum'>GBM with Enum Encoding</a>
43. <a href='#gbmTarget'>GBM with sort_by_response Encoding</a>
44. <a href='#gbmTargetDev'>GBM with Target Encoding Channel/Device Analysis</a>


#### 5. <a href='#visualization'>Visualization</a>
51. <a href='#varimp'>Variable Importance Plot</a>
52. <a href='#pdp'>Partial Dependency Plot</a>
53. <a href='#pdp2dim'>Two Variable Partial Dependency Plot</a>
54. <a href='#Treeplot'>Major Decision Trees Plot</a>

#### 6. <a href='#Out-Of-Sample Walk Forward Testing'>Out-Of-Sample Walk Forward Testing</a>


#### 7. <a href='#conclusion'>Conclusion</a>

In [2]:
import pandas as pd
import json
import random
import csv
import numpy as np
from datetime import datetime

In [3]:
import seaborn
import matplotlib.pyplot as plt
%matplotlib inline
seaborn.set(rc={'figure.figsize':(15,12)})

In [4]:
import subprocess
from IPython.display import Image

In [5]:
def json_to_series(text):
    keys, values = zip(*[item for item in json.loads(text).items()])
    return pd.Series(values, index=keys)

<a id='Sampling'></a>
# 0.Sample 10% of data as training set
<a href='#top'>back to top</a>

In [39]:
filename = "../data/train.csv"

In [40]:
n = sum(1 for line in open(filename,encoding="utf8")) - 1 

In [71]:
df = pd.read_csv(filename,nrows=int(n/10))

/Users/Shuxinxu/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<a id='clean'></a>
# 1.Data Cleaning

<a id='cleanID'></a>
### 1.1 Clean ID
<a href='#top'>back to top</a>

<a id='cleanTotals'></a>
### 1.2 Clean Totals
<a href='#top'>back to top</a>

In [72]:
df = pd.concat([df, df['totals'].apply(json_to_series)], axis=1)

df.drop(['totals'],axis=1,inplace=True)

In [73]:
def convert_int(df,cols):
    for col in cols:
        df[col] = df[col].map(lambda x: int(x) if not pd.isnull(x) else 0)
    return df

In [74]:
df = convert_int(df,['pageviews','newVisits','visits','bounces','transactionRevenue'])

In [75]:
df['log_transactionRevenue'] = df['transactionRevenue'].map(lambda x:np.log(x+1))

<a id='cleanTS'></a>
### 1.3 Clean Time Series
<a href='#top'>back to top</a>

In [76]:
df["raw_visitStartTime"] = df["visitStartTime"].map(lambda x:datetime.utcfromtimestamp(x))

df["visitStartTime"] = df["visitStartTime"].map(lambda x:datetime.utcfromtimestamp(x).hour)

In [77]:
df.sort_values(by=['fullVisitorId','raw_visitStartTime'],inplace=True)

df['nvisits'] = df.groupby(by='fullVisitorId')['fullVisitorId'].cumcount()+1

In [78]:
def get_diff_time(x):
    
    return int(x.seconds//3600)+x.days*24

In [79]:
df['pre_visitStartTime'] = df.groupby(by='fullVisitorId')['raw_visitStartTime'].shift(1)

df['diff_lastVisitTime'] = (df['raw_visitStartTime']-df['pre_visitStartTime'])

df['diff_lastVisitTime'] = df['diff_lastVisitTime'].map(lambda x: get_diff_time(x) if not pd.isnull(x) else np.NaN)

In [80]:
df['week'] = df['raw_visitStartTime'].map(lambda x: x.isocalendar()[1])

df['day_of_week'] = df['raw_visitStartTime'].map(lambda x: x.isocalendar()[2])

In [81]:
df.drop(['date'],axis=1,inplace=True)

<a id='cleanLoc'></a>
### 1.4 Clean Location
<a href='#top'>back to top</a>

In [82]:
df = pd.concat([df, df['geoNetwork'].apply(json_to_series)], axis=1)

df.drop(['geoNetwork'],axis=1,inplace=True)

<a id='cleanDev'></a>
### 1.5 Clean Device
<a href='#top'>back to top</a>

In [83]:
df = pd.concat([df, df['device'].apply(json_to_series)], axis=1)

df.drop(['device'],axis=1,inplace=True)

<a id='cleanCust'></a>
### 1.6 Clean Traffic Source
<a href='#top'>back to top</a>

In [88]:
df = pd.concat([df, df['trafficSource'].apply(json_to_series)], axis=1)

df.drop(['trafficSource'],axis=1,inplace=True)

In [98]:
df['adwordsClickInfo'] = df['adwordsClickInfo'].map(lambda x: str(x).replace("'","\""))

In [ ]:
df = pd.concat([df, df['adwordsClickInfo'].apply(json_to_series)], axis=1)

df.drop(['adwordsClickInfo'],axis=1,inplace=True)

<a id='feature-eng'></a>
# 2. Feature Cleaning
<a href='#top'>back to top</a>

In [101]:
ID = ["fullVisitorId","visitId"]

<a id='dropUnif'></a>
### 2.1 Drop uninformative categorical varibles
<a href='#top'>back to top</a>

In [102]:
for col in df.columns:
    cn = df[col].value_counts()
    if cn.shape[0]==1:
        df.drop(col,axis=1,inplace=True)

<a id='dropMissing'></a>
### 2.2 Drop variables with too many missing values
<a href='#top'>back to top</a>

In [105]:
df.replace("not available in demo dataset",np.NaN,inplace=True)

df.replace("NaN",np.NaN,inplace=True)

df.replace("(not set)",np.NaN,inplace=True)

In [106]:
na_df = np.sum(df.isna(),axis=0)/df.shape[0]
na_df[na_df>0.1]

sessionQualityDim     0.53872
timeOnSite            0.49974
transactions          0.98998
pre_visitStartTime    0.90097
diff_lastVisitTime    0.90097
region                0.57240
metro                 0.77939
city                  0.58195
networkDomain         0.28537
index                 0.20214
value                 0.20214
dtype: float64

In [107]:
df.drop(['index',"isMobile","transactions","metro"],axis=1,inplace=True)

<a id='encodeCat'></a>
### 2.3 Encode Categorical Variables
<a href='#top'>back to top</a>

In [108]:
import category_encoders as ce

<a id='encodeCat1'></a>
### 2.3.1 Encode Categorical Variables -- networkDomain
<a href='#top'>back to top</a>

In [109]:
df.loc[df['networkDomain'].isna(),'networkDomain'] = 'unknown.unknown'

df["raw_networkDomain"] = df["networkDomain"]

In [110]:
min_sample_leaf = int(df.shape[0]*0.005)

encoder = ce.TargetEncoder(cols=["networkDomain"],min_samples_leaf=min_sample_leaf,smoothing=1)

In [111]:
encoder.fit(df.loc[:,~df.columns.isin(["transactionRevenue","log_transactionRevenue"])], df["log_transactionRevenue"])

TargetEncoder(cols=['networkDomain'], drop_invariant=False,
       handle_unknown='impute', impute_missing=True, min_samples_leaf=500,
       return_df=True, smoothing=1.0, verbose=0)

In [112]:
df_x = encoder.transform(df.loc[:,~df.columns.isin(["transactionRevenue","log_transactionRevenue"])])

In [113]:
df['networkDomain'] = df_x['networkDomain']

In [114]:
cl='networkDomain'
    
_dict={}

for k,g in df.groupby(by=cl):

    if len(g['raw_'+cl].unique())<=10:
        _dict[k] = list(g['raw_'+cl].unique())
    else:
        _dict[k] = ['Others']
    
networkDomain_dict = _dict 

<a id='encodeCat2'></a>
### 2.3.2 Encode Categorical Variables -- operating Systems
<a href='#top'>back to top</a>

In [115]:
df['raw_operatingSystem'] = df['operatingSystem']
df['raw_browser'] = df['browser']
df['raw_region'] = df['region']
df['raw_city'] = df['city']
df['raw_country'] = df['country']

In [116]:
cols = ["operatingSystem","browser","region","city","country"]

min_sample_leaf = int(df.shape[0]*0.005)

encoder = ce.TargetEncoder(cols=cols,
                           min_samples_leaf=min_sample_leaf,smoothing=1)

In [117]:
encoder.fit(df.loc[:,~df.columns.isin(["transactionRevenue","log_transactionRevenue"])], df["log_transactionRevenue"])

TargetEncoder(cols=['operatingSystem', 'browser', 'region', 'city', 'country'],
       drop_invariant=False, handle_unknown='impute', impute_missing=True,
       min_samples_leaf=500, return_df=True, smoothing=1.0, verbose=0)

In [118]:
df_x = encoder.transform(df.loc[:,~df.columns.isin(["transactionRevenue","log_transactionRevenue"])])

In [119]:
for col in cols:
    df[col] = df_x[col]

In [120]:
cl='operatingSystem'
    
_dict={}

for k,g in df.groupby(by=cl):

    if len(g['raw_'+cl].unique())<=10:
        _dict[k] = list(g['raw_'+cl].unique())
    else:
        _dict[k] = ['Others']
    
operatingSystem_dict = _dict 

In [121]:
cl='browser'
    
_dict={}

for k,g in df.groupby(by=cl):

    if len(g['raw_'+cl].unique())<=10:
        _dict[k] = list(g['raw_'+cl].unique())
    else:
        _dict[k] = ['Others']
    
browser_dict = _dict 

In [122]:
cl='region'
    
_dict={}

for k,g in df.groupby(by=cl):

    if len(g['raw_'+cl].unique())<=10:
        _dict[k] = list(g['raw_'+cl].unique())
    else:
        _dict[k] = ['Others']
    
region_dict = _dict 

In [123]:
cl='city'
    
_dict={}

for k,g in df.groupby(by=cl):

    if len(g['raw_'+cl].unique())<=10:
        _dict[k] = list(g['raw_'+cl].unique())
    else:
        _dict[k] = ['Others']
    
city_dict = _dict 

In [124]:
cl='country'
    
_dict={}

for k,g in df.groupby(by=cl):

    if len(g['raw_'+cl].unique())<=10:
        _dict[k] = list(g['raw_'+cl].unique())
    else:
        _dict[k] = ['Others']
    
country_dict = _dict 

In [125]:
df.to_csv("train.csv",index=False)